## algorithm design and anlysis-2025 spring  homework 4
**Deadline**：2025.5.14

**name**:


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

## 问题 1 
**最小生成树（Minimum Spanning Tree）**

设  **G**  为一个带权重的连通无向图，且所有边的权重均不相等。令$e_i$ 为权重第 $i$ 小的边。最小生成树（MST）是否必须包含 $e_1$ ? 同理，是否必须包含 $e_2$ 和 $e_3$ ? 若必须包含，请给出证明；否则，请构造反例。需从基本原理论证，不能依赖割引理(cut lemma) 或 Prim/Kruskal算法的正确性。


answer:最小生成树的基本概念：MST 是一棵连接图中所有顶点的树，且没有环，边的总权值最小。
1. 边𝑒1权重最小：假设 MST 𝑇不含 𝑒1。𝑇连通，存在 u 到 v 的路径 P。加入e1，形成环 𝐶={𝑒1}∪𝑃移除 P 上一条边 𝑒′≠𝑒1，得新树 𝑇′。𝑇′连通（环保证连通性），权重 𝑤(𝑇′)=𝑤(𝑇)+𝑤(𝑒1)−𝑤(𝑒′)。
因 𝑤(𝑒1)<𝑤(𝑒′)，𝑤(𝑇′)<𝑤(𝑇)，矛盾（T 应权重最小）。故 𝑒1必须在 MST 中
2. 边e2权重次小：构造反例，存在 MST 不含 e 2。
顶点：{𝐴,𝐵,𝐶,𝐷}。边：e1=(A,B),w=1;e2=(B,C),w=2; e 3 =(A,C),w=3; e4=(C,D),w=4; e5=(A,D),w=5.
MST：T={e1,e3,e4}={(A,B),(A,C),(C,D)}，权重 1+3+4=8。
另一生成树：
{e1,e2,e4}，权重 1+2+4=7，是 MST，含𝑒2
但若选择其他边组合，𝑒2不一定在最小权重生成树中（依赖图结构）。
表明 e2不必须包含。
3.边 e3权重第三小：
类似 𝑒2，构造反例。使用与 𝑒2相同的图。
MST：{e1,e2,e4}，权重 1+2+4=7，不含 𝑒3 。
其他生成树可能包含 𝑒3，但 MST 可不含 𝑒3。
故 𝑒3不必须包含。

In [1]:
class UnionFind:
    def __init__(self, size):
        self.root = list(range(size))
        self.height = [0] * size
    
    def find(self, x):
        # Path compression: make each node point to the root
        while self.root[x] != x:
            self.root[x] = self.root[self.root[x]]
            x = self.root[x]
        return x
    
    def merge(self, x, y):
        # Union by height
        rx, ry = self.find(x), self.find(y)
        if rx == ry:
            return False
        if self.height[rx] < self.height[ry]:
            rx, ry = ry, rx
        self.root[ry] = rx
        if self.height[rx] == self.height[ry]:
            self.height[rx] += 1
        return True

def build_mst(vertex_count, edge_list):
    # Sort edges by weight
    sorted_edges = sorted(edge_list, key=lambda e: e[2])
    uf = UnionFind(vertex_count)
    mst = []
    # Apply Kruskal's algorithm
    for u, v, w in sorted_edges:
        if uf.merge(u, v):
            mst.append((u, v, w))
    return mst

def check_edge_in_mst(vertex_count, edge_list, target_edge):
    u, v, target_weight = target_edge
    # Step 1: Build MST and check if target edge is included
    mst = build_mst(vertex_count, edge_list)
    mst_edges = {(min(x, y), max(x, y)) for x, y, _ in mst}
    target = (min(u, v), max(u, v))
    is_in_mst = target in mst_edges
    
    # Step 2: Check if edge must be in MST (using replacement logic)
    if is_in_mst:
        # If in MST, check if replacing it increases weight
        for other_edge in edge_list:
            if other_edge == target_edge:
                continue
            x, y, w = other_edge
            if w < target_weight:
                # Simulate adding target_edge to MST without it
                temp_edges = [e for e in mst if (min(e[0], e[1]), max(e[0], e[1])) != target]
                uf = UnionFind(vertex_count)
                for e in temp_edges:
                    uf.merge(e[0], e[1])
                # If u, v disconnected, target_edge may be necessary
                if uf.find(u) != uf.find(v):
                    return "YES"  # No lighter edge connects the components
    else:
        # If not in MST, it might still be in another MST
        return "NO"  # Since it's not in this MST, it's not definitely included
    
    # Step 3: If no definitive result, edge inclusion is uncertain
    return "MAYBE"

def process_mst_queries(vertex_count, edge_count, edge_list, query_edges):
    # Process each query edge and return results
    results = []
    for edge in query_edges:
        result = check_edge_in_mst(vertex_count, edge_list, edge)
        results.append(result)
    return results

# Test case
if __name__ == "__main__":
    vertex_count, edge_count = 4, 5
    edges = [(0, 1, 1), (1, 2, 2), (2, 3, 3), (3, 0, 4), (0, 2, 5)]
    queries = [(0, 1, 1), (1, 2, 2), (0, 2, 5)]  # e1, e2, e3
    
    results = process_mst_queries(vertex_count, edge_count, edges, queries)
    for i, result in enumerate(results, 1):
        print(f"Edge e{i}: {result}")

Edge e1: MAYBE
Edge e2: YES
Edge e3: NO


## 问题 2 
**瓶颈生成树（Bottleneck Spanning Tree）**

带有权重的无向图 $G(V,E,w)$ 的瓶颈生成树，表现为：在所有生成树中，最大权重边的权重值最小。即，BST $T$ 最小化瓶颈损失 $c(T)=max_{e \in T}{w(e)}$。

1. 证明 $G$ 的每一个最小生成树（MST）都是瓶颈生成树（BST）
2. 设计一个线性时间复杂度的算法：， 对于一个图 $G(V,E,w)$ 和一个整数 $b$，判断图 $ G$ 是否存在一个瓶颈生成树，其最大权重边的权重不超过 $b$，分析算法设计思路，并基于python编程实现。
3. 设计一个线性时间复杂度的算法：对于给定的图 $G(V,E,w)$，找到其瓶颈生成树，分析算法设计思路，并基于python编程实现。

idea：我们通过反证法证明任意MST 都是 BST。设 𝑇𝑀𝑆𝑇T MST是一棵 MST，𝑇𝐵𝑆𝑇T BST 是一棵 BST，𝑐(𝑇𝐵𝑆𝑇)=𝑤𝑚𝑖𝑛是所有生成树最大边权的最小值。
假设 𝑇MST不是 BST，即 𝑐(𝑇𝑀𝑆𝑇)>𝑤𝑚𝑖𝑛。则 𝑇 MST中存在边 𝑒𝑚𝑎𝑥满足 𝑤(𝑒𝑚𝑎𝑥)>𝑤𝑚𝑖𝑛。𝑇BST的所有边权 𝑤(𝑒)≤𝑤𝑚𝑖𝑛w(e)≤w min，且 𝑇𝐵𝑆𝑇T BST是生成树。比较两棵树的性质：>w min 构造过程：这两个思路基于图的生成树性质，避免依赖复杂引理，直接从定义出发，符合问题要求。如果 c(TMST)>wmin ，可以用 TBST T_{BST} TBST​ 替换 TMST ，因为 TBST TBST连通且边权上限更低。由于 TMST 是总权值最小的生成树，若 TBST 的总权值小于 TMST （可能因 wmin限制），则矛盾。结论：c(TMST)≤wmin 必成立，否则 TMST 不是最小权值生成树。由于 wmin 是所有生成树 c(T) 的最小值，c(TMST)=wmin ，故 TMST 也是 BST。

idea：2. 思路：需要判断是否存在一棵生成树，其最大边权不超过 b 。这可以通过图的连通性检验实现。移除图 G 中所有权值 >b 的边，得到子图 G′。检查 G′ 是否仍连通（即所有顶点可达）。 如果 G′连通，则存在生成树仅包含边权 ≤b 的边，且其最大边≤b。如果 G′ 不连通，则至少有一对顶点无法通过≤b 的边连接，任何生成树必包含权值>b的边，最大边权>b。使用深度优先搜索（DFS）或并查集遍历  G′ 的连通分量。若所有顶点在同一连通分量中，返回“存在”；否则，返回“不存在”

In [2]:
from collections import defaultdict, deque

def can_form_bottleneck_spanning_tree(num_vertices, edge_list, max_weight):
    # B如果权重小于最大权重
    graph = defaultdict(list)
    for src, dst, weight in edge_list:
        if weight <= max_weight:
            graph[src].append(dst)
            graph[dst].append(src)
    
    # BFS to 检查链接
    seen = [False] * num_vertices
    queue = deque([0])  
    seen[0] = True
    
    while queue:
        current = queue.popleft()
        for neighbor in graph[current]:
            if not seen[neighbor]:
                seen[neighbor] = True
                queue.append(neighbor)
    
    
    return all(seen)

# 时间复杂度: O(|V| + |E|)

In [2]:
from collections import defaultdict, deque

# 思路：根据 MST 即 BST 的性质，通过二分查找找到所有生成树中最大边权的最小值，并构造相应生成树
def get_bottleneck_spanning_tree(vertex_count, edge_data):
    # 提取所有唯一边权并排序
    unique_weights = sorted(set(weight for _, _, weight in edge_data))
    if not unique_weights:
        return None, float('inf')  # 无边的情况
    
    # 二分查找最小瓶颈值
    low, high = 0, len(unique_weights) - 1
    min_bottleneck = float('inf')
    
    while low <= high:
        mid = (low + high) // 2
        threshold = unique_weights[mid]
        
        # 构建包含权值 <= threshold 的子图
        graph = defaultdict(list)
        for src, dest, weight in edge_data:
            if weight <= threshold:
                graph[src].append(dest)
                graph[dest].append(src)
        
        # BFS 检查子图连通性
        explored = [False] * vertex_count
        queue = deque([0])
        explored[0] = True
        
        while queue:
            current_vertex = queue.popleft()
            for next_vertex in graph[current_vertex]:
                if not explored[next_vertex]:
                    explored[next_vertex] = True
                    queue.append(next_vertex)
        
        if all(explored):  # 子图连通，更新最小瓶颈值
            min_bottleneck = threshold
            high = mid - 1
        else:
            low = mid + 1
    
    # 若无有效瓶颈值，返回空
    if min_bottleneck == float('inf'):
        return None, min_bottleneck
    
    # 构建包含最佳瓶颈值的子图，并生成生成树
    graph = defaultdict(list)
    for src, dest, weight in edge_data:
        if weight <= min_bottleneck:
            graph[src].append((dest, weight))
            graph[dest].append((src, weight))
    
    # BFS 构造生成树
    explored = [False] * vertex_count
    spanning_tree = []
    queue = deque([(0, -1)])  # (当前顶点, 父顶点)
    explored[0] = True
    
    while queue:
        current_vertex, parent = queue.popleft()
        for next_vertex, weight in graph[current_vertex]:
            if not explored[next_vertex] and next_vertex != parent:
                spanning_tree.append((current_vertex, next_vertex, weight))
                explored[next_vertex] = True
                queue.append((next_vertex, current_vertex))
    
    return spanning_tree, min_bottleneck

# 测试用例
vertex_num = 4
edge_list = [(0, 1, 1), (1, 2, 2), (2, 3, 3), (3, 0, 4), (0, 2, 5)]
tree_result, bottleneck_value = get_bottleneck_spanning_tree(vertex_num, edge_list)
print("瓶颈生成树边:", tree_result)
print("瓶颈值 c(T):", bottleneck_value)

# 时间复杂度：O(|E| log |E|)

瓶颈生成树边: [(0, 1, 1), (1, 2, 2), (2, 3, 3)]
瓶颈值 c(T): 3


## 问题 3

**道路网（Road Network）**

假设有一个以图 $ G(V, E, l) $ 表示的道路网络，连接了一组城市 $ V $。我们假设该网络是有向的，并且每条道路 $(u, v) \in E$ 都有一个非负的长度 $ l(u, v) $。一条新的道路即将被建造，因此有一个列表 $ E' $ 包含它可以连接的城市对。每对 $(u, v) \in E'$ 都有一个对应的长度 $ l'(u, v) $。我们希望选择一对城市，使得两个城市 $ s, t \in V $ 之间的距离减少最大。请为此问题编写一个高效的算法，并详细解释算法的正确性和复杂度。


In [3]:
from heapq import heappush, heappop
from collections import deque

def is_path_within_limit(vertex_count, edge_list, start, end, max_length):
    # 构建邻接表，存储 (邻接顶点, 边长)
    neighbors = {i: [] for i in range(vertex_count)}
    for u, v, length in edge_list:
        neighbors[u].append((v, length))
    
    # 优先队列：(路径长度, 当前顶点)
    queue = [(0, start)]  # (总长度, 顶点)
    # 记录到每个顶点的最短路径长度
    shortest_paths = {v: float('inf') for v in range(vertex_count)}
    shortest_paths[start] = 0
    
    while queue:
        curr_length, curr_vertex = heappop(queue)
        
        # 如果到达目标且路径长度合法
        if curr_vertex == end and curr_length <= max_length:
            return True
        
        # 如果当前路径已超过限制，跳过
        if curr_length > max_length:
            continue
        
        # 如果当前路径不是最优，跳过
        if curr_length > shortest_paths[curr_vertex]:
            continue
        
        # 扩展到邻接顶点
        for next_vertex, edge_length in neighbors[curr_vertex]:
            new_length = curr_length + edge_length
            if new_length < shortest_paths[next_vertex]:
                shortest_paths[next_vertex] = new_length
                heappush(queue, (new_length, next_vertex))
    
    return False

# 测试用例
num_vertices = 4
edge_data = [(0, 1, 1), (1, 2, 2), (2, 3, 3), (0, 2, 5), (1, 3, 6)]  # (u, v, 边长)
start_point, end_point, length_limit = 0, 3, 4

result = is_path_within_limit(num_vertices, edge_data, start_point, end_point, length_limit)
print("能否从 s 到达 t 且路径长度 <= L:", result)

# 时间复杂度：O(|E| log |V|)

能否从 s 到达 t 且路径长度 <= L: False


## 问题 4

**逃离问题**

一个 $ n \times n $ 的网格是一个无向图，由 $ n $ 行和 $ n $ 列的顶点组成，如下图所示。我们用 $(i,j)$ 表示第 $ i $ 行和第 $ j $ 列的顶点。除了边界顶点，网格中的所有顶点都有四个邻居，即满足 $ i = 1, i = n, j = 1 $ 或 $ j = n $ 的点 $(i,j)$。

给定网格中的 $ m \leq n^2 $ 个起点 $(x_1, y_1), (x_2, y_2), \cdots , (x_m, y_m)$，逃离问题是确定是否存在 $ m $ 条顶点不相交的路径（即路径之间不相交），从这些起点到边界上的任意 $ m $ 个不同点。例如，图1中的网格存在逃离。

(1) 该问题可以看作是一个最大流问题。考虑一个流网络，其中顶点和边都有容量。也就是说，进入任何给定顶点的总正流量受到容量限制。证明在具有边和顶点容量的网络中确定最大流可以简化为在具有可比大小的普通流网络上的最大流问题。更准确地说，你需要将一个具有顶点和边容量的网络 $ G = (V,E) $ 转换为另一个仅具有边容量的网络 $ G' = (V', E') $，使得两个网络上的最大流相同，并且你构建的新网络具有 $ V' = O(V) $ 个顶点和 $ E' = O(E) $ 条边。你可以假设网络是连通的。

(2) 描述一个解决逃离问题的高效算法，并分析其运行时间。


<div align="center"> <img alt="图片" src="./fig/escepe-p.png"> </div>
<center> 图1. 逃脱问题网格，起始顶点为黑色，其他网格顶点为白色</center>

idea：